In [1]:
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import json
import dataset
import algorithm

In [2]:
CONFIG = json.load(open('config.json'))

In [3]:
Dataset = dataset.download_dataset(CONFIG['data']['filename'])

In [4]:
data = dataset.get_data(Dataset)
data.head()

,area,material,condicion,anio_construccion,anio_remodelacion,sotano,calefaccion,aire_acondicionado,area_construida_piso_1,area_construida,banios,banios_sin_ducha,dormitorios,chimeneas,area_garage,area_piscina,precio
0,8450,7,5,2003,2003,1,1,1,856,1710,2,1,3,0,548,0,208500
1,9600,6,8,1976,1976,1,1,1,1262,1262,2,0,3,1,460,0,181500
2,11250,7,5,2001,2002,1,1,1,920,1786,2,1,3,1,608,0,223500
3,9550,7,5,1915,1970,2,1,1,961,1717,1,0,3,1,642,0,140000
4,14260,8,5,2000,2000,1,1,1,1145,2198,2,1,4,1,836,0,250000


In [5]:
normalize_data = dataset.Normalizer.transform(data)

In [6]:
data_x, data_y = dataset.split_data_x_y(normalize_data)

In [7]:
data_x_train, data_x_test = dataset.split_data_train_test(data_x, CONFIG["data"]["train_split_size"])
data_y_train, data_y_test = dataset.split_data_train_test(data_y, CONFIG["data"]["train_split_size"])

In [8]:
dataset_train = dataset.TimeSeriesDataset(data_x_train, data_y_train)
dataset_test = dataset.TimeSeriesDataset(data_x_test, data_y_test)

In [9]:
# DataLoader => Devuelve un conjunto de iteraciones, cada iteracion contiene un arreglo de 'batch_size' registros
train_dataloader = DataLoader(dataset_train, CONFIG["training"]["batch_size"], shuffle=True)
test_dataloader = DataLoader(dataset_test, CONFIG["training"]["batch_size"], shuffle=True)

In [10]:
model = algorithm.Model(config_model=CONFIG["model"], config_training=CONFIG["training"])

In [11]:
# Entrenamiento del modelo
for epoch in range(CONFIG["training"]["num_epoch"]):
    loss_train, lr_train = model.run_epoch(train_dataloader, is_training=True)
    loss_test, lr_test = model.run_epoch(test_dataloader)
    if epoch == 0 or ((epoch+1)%10 == 0):
        print('Epoch[{}/{}] | loss train:{:.6f}, test:{:.6f} | lr:{:.6f}'
              .format(epoch+1, CONFIG["training"]["num_epoch"], loss_train, loss_test, lr_train))

Epoch[1/100] | loss train:0.004173, test:0.001194 | lr:0.010000
Epoch[10/100] | loss train:0.000621, test:0.000292 | lr:0.010000
Epoch[20/100] | loss train:0.000557, test:0.000623 | lr:0.010000
Epoch[30/100] | loss train:0.000511, test:0.000293 | lr:0.010000
Epoch[40/100] | loss train:0.000469, test:0.000268 | lr:0.010000
Epoch[50/100] | loss train:0.000621, test:0.000337 | lr:0.010000
Epoch[60/100] | loss train:0.000486, test:0.000319 | lr:0.010000
Epoch[70/100] | loss train:0.000483, test:0.000506 | lr:0.010000
Epoch[80/100] | loss train:0.000502, test:0.000228 | lr:0.010000
Epoch[90/100] | loss train:0.000696, test:0.000263 | lr:0.010000
Epoch[100/100] | loss train:0.000563, test:0.000287 | lr:0.010000


In [12]:
# Verificacion de la prediccion del modelo
error = 0
i=0

for idx, (x, y) in enumerate(test_dataloader):
    i+=1
    x = x.to(CONFIG["training"]["device"])
    out = model(x)
    out_vector = out.detach().numpy()
    y_vector = y.numpy()
    for i in range(len(y_vector)):
        price_predict = dataset.Normalizer.inverse_transform(out_vector[i], CONFIG["model"]["input_size"]+1)
        price_real = dataset.Normalizer.inverse_transform(y_vector[i], CONFIG["model"]["input_size"]+1)
        porcentage = price_predict/price_real
        error += abs(1-porcentage)
        print('Price predict: {0}, Price real: {1}, Diferencia: {2} ({3})'.format(price_predict, price_real, price_predict-price_real, (porcentage)))
    
    print("margen de error {}%".format(round(100 * error / i, 2)))
    break

Price predict: 178227, Price real: 180000, Diferencia: -1773 (0.99015)
Price predict: 163665, Price real: 165000, Diferencia: -1335 (0.991909090909091)
Price predict: 166423, Price real: 193000, Diferencia: -26577 (0.8622953367875648)
Price predict: 103223, Price real: 125000, Diferencia: -21777 (0.825784)
Price predict: 113103, Price real: 138500, Diferencia: -25397 (0.8166281588447654)
Price predict: 169505, Price real: 185000, Diferencia: -15495 (0.9162432432432432)
Price predict: 104692, Price real: 120500, Diferencia: -15808 (0.8688132780082988)
Price predict: 109533, Price real: 112000, Diferencia: -2467 (0.9779732142857143)
Price predict: 112509, Price real: 91500, Diferencia: 21009 (1.2296065573770492)
Price predict: 185991, Price real: 179600, Diferencia: 6391 (1.0355846325167037)
Price predict: 107449, Price real: 119000, Diferencia: -11551 (0.9029327731092437)
Price predict: 107569, Price real: 104900, Diferencia: 2669 (1.025443279313632)
Price predict: 172907, Price real: 2

In [13]:
import torch
d = np.array([[8635, 5, 5, 1948, 2001, 'TA', 'GasA', 'Y', 1072, 1285,1, 0, 2, 0, 240, 0, 0]])
d = pd.DataFrame(d, columns=['area', 'material', 'condicion', 'anio_construccion', 'anio_remodelacion', 'sotano', 'calefaccion',  'aire_acondicionado', 'area_construida_piso_1', 'area_construida', 'banios', 'banios_sin_ducha', 'dormitorios', 'chimeneas', 'area_garage', 'area_piscina', 'precio'])
d = dataset.get_data(d)

d = dataset.Normalizer.transform(d)
info_x, _ = dataset.split_data_x_y(d)
x = torch.tensor(info_x).float().to(CONFIG["training"]["device"])
out = model(x)
out_vector = out.detach().numpy()
price_predict = dataset.Normalizer.inverse_transform(out_vector[0], CONFIG["model"]["input_size"]+1)
print(price_predict)

0
